# Primena **DBSCAN** algoritma na podatke o geografskoj lokaciji aerodroma

## Vršimo neophodne *import* - e

In [1]:
import pandas as pd # Pandas biblioteka za DataFrame strukturu
import numpy as np  # Numpy biblioteka
from matplotlib import pyplot as plt # Biblioteka za vizuelizaciju
from mpl_toolkits.basemap import Basemap # Biblioteka za vizuelizaciju mape
from copy import deepcopy
import math

from sklearn.preprocessing import MinMaxScaler # Skaliranje vrednosti atributa na isti interval, (0,1)
from sklearn.metrics import silhouette_score # Senka koeficijent, mera kvaliteta klasterovanja
from sklearn.cluster import DBSCAN # DBSCAN algoritam za klasterovanje

## Učitavanje podataka i upoznavanje sa njima

In [2]:
# Napomena: Ovo su podaci već delom preprocesirani u IBM-SPSS modeleru
df = pd.read_csv("../Data/airports_PreprocessedData.csv")
df.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source,Altitude_CATEGORY,Coord_X,Coord_Y,Coord_X_Nearest_End
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10.0,U,Pacific/Port_Moresby,airport,OurAirports,2,16158.656679,-675.910224,3840.837979
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10.0,U,Pacific/Port_Moresby,airport,OurAirports,1,16202.779039,-578.707337,3796.715620
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10.0,U,Pacific/Port_Moresby,airport,OurAirports,2,16036.849596,-647.581008,3962.645063
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10.0,U,Pacific/Port_Moresby,airport,OurAirports,1,16306.913146,-730.158416,3692.581512
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10.0,U,Pacific/Port_Moresby,airport,OurAirports,1,16361.818285,-1049.523652,3637.676374


## Vizuelizacija - pre primene algoritma

In [3]:
plt.rcParams["font.size"] = 30

fig = plt.gcf()
fig.set_size_inches(20, 20) # Veličina mape

m = Basemap(projection='cyl', \   # Biramo cilindričnu projekciju
            llcrnrlat=-80, urcrnrlat=80, \
            llcrnrlon=-180, urcrnrlon=180, \
            lat_ts=20, \
            resolution='c')

m.bluemarble(scale=0.2)   # full scale will be overkill
m.drawcoastlines(color='white', linewidth=0.2)  # add coastlines

x, y = m(list(df['Longitude']),list(df['Latitude']))  # transform coordinates

colors = ['red','green','skyblue'] # Aerodrome bojimo na mapi u zavisnosti od visinske kategorije
category_altitude = list(df['Altitude_CATEGORY'])

plt.title("Aerports")
for i in range(0,len(x)):
    plt.scatter(x[i],y[i], color = colors[category_altitude[i] - 1])

plt.show()

SyntaxError: unexpected character after line continuation character (<ipython-input-3-cd3da5b79e27>, line 6)

## Preprocesiranje podataka
### Izdvajanje karakteristika po kojima vrsimo klasterovanje
#### (za početak ćemo vršiti klasterovanje po atributima 
####  *Longitude* , *Latitude* i *Altitude* 

In [ ]:
features = df.columns[6:8]
df[features].head()

## Preprocesiranje podataka
### Skaliranje raspona vrednosti svih atributa po kojima
### vršimo skaliranje na interval $(0,1)$

In [ ]:
scaler = MinMaxScaler()
scaler.fit(df[features]) # Za svaki atribut nalazi najveću i najmanju vrednost, na osnovu njih će skalirati
x = pd.DataFrame(scaler.transform(df[features])) # Skaliranje vrednosti
x.columns = features
x.head()

## Primena algoritma **DBSCAN** za različite vrednosti parametara i upoređivanje rezultata

  Dobijena vizuelizacija nam može pomoći pri proceni za odabir parametara, $\varepsilon$ i minimalnog broja suseda. Više puta ćemo pokušavati sa različitim kombinacijama. 

In [ ]:
colors = ['red','deepskyblue','gold','lightgreen','palevioletred','purple','orangered',
          'blueviolet','darkblue','cadetblue','indigo','tomato','mediumaquamarine',
          'olive','c','maroon','darkseagreen','cornflowerblue','orchid','lightcoral',
          'lavender','slateblue','yellowgreen','forestgreen','antiquewhite', 'sienna',
          'darkcyan','deeppink','thistle','plum','powderblue','sandybrown','m',
          'chartreuse','lemonchiffon','mistyrose','darkmagenta','brown','peru',
          'chocolate','royalblue']

# Prvo ćemo napisati funkciju za treniranje modela, ispis rezultata i prikaz
# df - originalni DataFrame koji obrađujemo
# x - DataFrame sa skaliranim vrednostima i izdvojenim karakteristikama, eps i min_samples
def model_DBSCAN(df,x,eps,min_samples):
    est = DBSCAN(eps = eps,min_samples = min_samples) # Pravimo instancu modela
    est.fit(x) # Treniramo model
    
    df['labels'] = est.labels_ # Dodeljujemo dobijene klastere
    
    senka_koeficijent = silhouette_score(x, est.labels_)
    # Ispisujemo kvalitet klasterovanja
    print("** Dobijeni senka koeficijent je {} ".format(senka_koeficijent))
    if senka_koeficijent <= 0.0:
        print("-- Los kvalitet klasterovanja (POOR)")
    
    elif senka_koeficijent <= 0.5:
        print("-- Nije dobar kvalitet klasterovanja (FAIR)")
    
    elif senka_koeficijent < 0.7:
        print("-- Dobar kvalitet klasterovanja (GOOD)")
        
    else:
        print("-- Najbolji kvalitet klasterovanja, klasteri su gusti i dobro grupisani (GOOD)")
    num_of_clusters = max(df['labels']) + 1
    
    # Iscrtavamo klastere na mapi
    
    figure = plt.figure(figsize= (20,9.5))
    for i in range(-1,num_of_clusters):
        
        if i == -1: # Da li trenutno crtamo šum
            label = "noise"
            color = "lightgray"
        else:
            label = "cluster %d"%i
            color = colors[i]
        
        # Elementi koji pripadaju klasteru koji trenutno crtamo
        current_cluster = df[df['labels'] == i]
        plt.scatter(current_cluster['Longitude'], current_cluster['Latitude'], 
                   color = color, label = label)
    plt.legend()
    plt.show()
    

In [ ]:
# Prva procena: eps = 0.05 , minimalan broj suseda = 100

model_DBSCAN(df = df,x = x,eps = 0.05, min_samples= 100)

### Analiza dosadašnjih rezultata i procene za dalje pokušaje

  Rezultat nije dobar kao sto smo očekivali. Veliki broj elemenata je proglašen šumom. S druge strane, klasteri nisu ni previše gusti.
  
  Na ovoj slici vidimo i problem sa našim podacima. U Evropi, Severnoj Americi, po obodima Južne Amerike i Afrike je prilično velika gustina aerodroma. Za razliku od tih delova sveta, u većem delu Azije nema toliko gustih delova koji bi obuhvatili ostale aerodrome u svoj klaster.
  
  **Novi pokušaj:** Ostavićemo trenutno isto $\varepsilon$, a pokušaćemo sa manjim brojem
  minimalnog broja suseda.

In [ ]:
# Nova procena: eps = 0.05, minimalan broj suseda = 90

model_DBSCAN(df = df, x = x, eps = 0.05, min_samples = 88)

In [ ]:
# Skoro sve tačke su ostale šum.
# Smanjujemo dalje minimalan_broj_suseda za isto epsilon
# Nova procena: eps = 0.1, minimalan_broj_suseda = 100

model_DBSCAN(df = df, x = x, eps = 0.05, min_samples = 80)

In [ ]:
# Možemo da primetimo nezanemarljivo povećanje senka-koeficijenta.
# Na osnovu te mere nam je klasterovanje nesto bolje, ali opet nije blizu dobrog.

# Minimalan broj suseda je bio dovoljno mali da neki granični elementi postanu elementi
# jezgra i time se dva klastera (aerodroma iz Afrike i Evrope) spoje u jedan, ali opet ne dovoljno mali da bi neki element
# sa severa Azije postao element jezgra.
# Svejedno, još ćemo smanjivati minimalan_broj_suseda da vidimo šta će dalje da se menja.

model_DBSCAN(df = df, x = x, eps = 0.05, min_samples = 68)

In [ ]:
# Samo neznatno raste senka koeficijent.
model_DBSCAN(df = df, x = x, eps = 0.05, min_samples = 59)

In [ ]:
model_DBSCAN(df = df, x = x, eps = 0.05, min_samples = 58)

### Analiza dosadasnjih rezultata i procene za dalje pokusaje

  U pretposlednja 2 pokusaja smo postigli najveci senka koeficijent i tu smo obuhvatili najveći broj elemenata koji su prethodno bili šum. U poslednjem pokušaju, gde je broj suseda bio smanjen za 1 u odnosu na prethodni je obuhvaćeno najviše elemenata i nastao je još jedan klaster. On je mnogo manji od ostalih i znatno je gušći (u odnosu na srednje gustine ostalih).
  
  Nažalost i najveća dobijena vrednost senka koeficijenta i dalje se ne graniči sa dobrim klasterovanjem. 
  
  Sledeće što možemo da pokušamo je da smanjimo i $\varepsilon$ i broj suseda.

In [ ]:
model_DBSCAN(df = df, x = x, eps = 0.045,min_samples = 80)

In [ ]:
model_DBSCAN(df = df, x = x, eps = 0.045,min_samples = 60)

In [ ]:
model_DBSCAN(df = df,x = x,eps = 0.02, min_samples = 25)

In [ ]:
# Nijednom nismo dobili bolje rezultate. Izvešćemo jos jedan pokušaj, sa većim epsilon.

model_DBSCAN(df = df,x = x,eps = 0.07, min_samples = 90)

## Zaključak: 
Jasno je da zbog različite gustine podataka ne možemo dobiti bolje klasterovanje primenom ovog algoritma. Iako smo samo probali sa izdvojenim 'Alitude', 'Latitude' i 'Longitude', jasno je da zbog istog problema ne bi bio bolji rezultat ni sa izdvojenim samo atributima 'Latitude' i 'Longitude', ni sa pristupom 'Longitude', 'Latitude' i 'Altitude_CATEGORY'. Ostavićemo primere i sa druga dva pristupa.

Pre nego što pokusamo sa izvršavanjem algoritma, moramo da preprocesiramo kategorički atribut 'Altitude_CATEGORY'. Imamo 3 kategorije i za svaku od njih pravimo jedan binarni atribut. Instanca neće imati 0 za tacno jedan atribut, koji odgovara vrednosti  atributa 'Altitude_CATEGORY'. Zbog računanja Euklidskog rastojanja i jednakog uticaja, umesto jedinice koristimo vrednost $\sqrt{\frac{1}{2}}$ .

In [ ]:
features = ['Longitude','Latitude'] # Izdvajamo i skaliramo vrednosti atributa Longitude i Latitude
x = df[features]
scaler = MinMaxScaler().fit(x)
x = pd.DataFrame(scaler.transform(x))
x.columns = features

x['Altitude_CATEGORY'] = df['Altitude_CATEGORY']
#x = deepcopy( df[features] )

x['CAT_1'] = 0.0   # Nova binarna 3 atributa koji će odgovarati svakoj od mogućih kategorija
x['CAT_2'] = 0.0
x['CAT_3'] = 0.0
fixed_value = math.sqrt(0.5) # Ne-nula vrednost za atribute

for i in range(1,4):  # Zamenjujemo nule na odgovarajućim vrednostima
    x.loc[x['Altitude_CATEGORY'] == i, 'CAT_%d'%i] = fixed_value

x.head()

In [ ]:
# Sada još da uklonimo atribut Altitude_CATEGORY
features = ['Latitude','Longitude','CAT_1','CAT_2','CAT_3']
x = x[features]
x.head()

In [ ]:
# Jedan od pokušaja
model_DBSCAN(df = df, x = x, eps = 0.05, min_samples = 59)

In [ ]:
model_DBSCAN(df = df,x = x,eps = 0.07, min_samples = 90)

In [ ]:
model_DBSCAN(df = df,x = x,eps = 0.05, min_samples= 100)

In [ ]:
# Klasterovanje na osnovu samo atributa Latitude i Longitude
x = x[['Longitude','Latitude']]
model_DBSCAN(df = df,x = x,eps = 0.07, min_samples = 90)